# Программирование для всех (основы работы с Python) 

*Алла Тамбовцева*

## Практикум по парсингу с `BeautifulSoup`

Мы уже немного познакомились со структурой html-страниц и поиском информации по тэгам, теперь попробуем выгрузить информацию из более интересной страницы, а точнее, с сайта [nplus1.ru](https://nplus1.ru/). Наша задача – выгрузить недавние новости в датафрейм `pandas`, чтобы потом сохранить все в файл Excel.

Для работы нам снова понадобится модуль `requests` для отправки запросов, для «подключения» к странице и получения ее содержимого в виде строки, и функция `BeautifulSoup` из библиотеки `bs4` для удобного поиска по полученной строке:

In [1]:
import requests
from bs4 import BeautifulSoup

Сохраним ссылку на главную страницу в переменную `main` и отправим запрос к ней с помощью функции `get()` из `requests`:

In [2]:
main = "https://nplus1.ru/"
page = requests.get(main)

Заберём исходный код страницы и преобразуем строку с ним в объект `BeautifulSoup`:

In [3]:
soup = BeautifulSoup(page.text)

Чтобы сгрузить все новости с главной страницы сайта, нужно собрать все ссылки на страницы с этими новостями. Ссылки в html-файле всегда заключены в тэг `<a></a>` и имеют атрибут `href`. Найдем кусочки кода HTML, соответствующие всем ссылкам на главной странице сайта:

In [4]:
links_raw = soup.find_all("a") 
links_raw[10:20]  # несколько штук для примера

[<a class="hover:underline transition-colors duration-75" href="https://nplus1.ru/search?tags=880">Астрономия</a>,
 <a class="hover:underline transition-colors duration-75" href="https://nplus1.ru/search?tags=768">Генетика</a>,
 <a class="hover:underline transition-colors duration-75" href="https://nplus1.ru/search?tags=890">Математика</a>,
 <a class="hover:underline transition-colors duration-75" href="https://nplus1.ru/search?tags=871">Космонавтика</a>,
 <a class="hover:underline transition-colors duration-75" href="https://nplus1.ru/search?tags=876">Археология</a>,
 <a class="hover:underline transition-colors duration-75" href="https://nplus1.ru/search?tags=775">Нейронауки</a>,
 <a class="hover:underline transition-colors duration-75" href="https://nplus1.ru/search?tags=767">На мышах</a>,
 <a class="hover:underline transition-colors duration-75" href="https://nplus1.ru/search?tags=771">Звук</a>,
 <a class="hover:underline transition-colors duration-75" href="https://nplus1.ru/search

Каждый элемент возвращаемого списка имеет тип `BeautifulSoup` и структуру, очень похожую на словарь. Например, ссылка `<a class="hover:underline transition-colors duration-75" href="/search/empty/768">Генетика</a>` изнутри выглядит как словарь следующего вида:

    {'href' : '/search/empty/768', 
     'class' : 'hover:underline transition-colors duration-75'}.
    
Как мы помним, значение по ключу из словаря можно вызвать с помощью метода `.get()`. Давайте извлечем значения по ключу `href` из каждого элемента списка `links`:

In [5]:
links = [li.get("href") for li in links_raw] 
links[10:20]  # несколько штук для примера

['https://nplus1.ru/search?tags=880',
 'https://nplus1.ru/search?tags=768',
 'https://nplus1.ru/search?tags=890',
 'https://nplus1.ru/search?tags=871',
 'https://nplus1.ru/search?tags=876',
 'https://nplus1.ru/search?tags=775',
 'https://nplus1.ru/search?tags=767',
 'https://nplus1.ru/search?tags=771',
 'https://nplus1.ru/search?tags=772',
 'https://nplus1.ru/search?tags=778']

Ссылок в списке выше много. Но нам нужны только новости – ссылки, которые начинаются с `https://nplus1.ru/news`. Создадим пустой список `news` и будем добавлять в него только ссылки, которые удовлетворяют этому условию.

In [6]:
news = []
for li in links:
    if "https://nplus1.ru/news/" in li:
        news.append(li)

In [7]:
news[0:10]

['https://nplus1.ru/news/2015/09/21/editor-thy-name',
 'https://nplus1.ru/news/2023/10/18/beware-ppi',
 'https://nplus1.ru/news/2023/10/21/frb-and-plasma',
 'https://nplus1.ru/news/2023/10/21/cognitive-behavior-therapy',
 'https://nplus1.ru/news/2023/10/21/pulsatile-hydrocortisone',
 'https://nplus1.ru/news/2023/10/21/daily-steps-prior-to-surgery',
 'https://nplus1.ru/news/2023/10/20/eye-ticking',
 'https://nplus1.ru/news/2023/10/20/python-ate-python',
 'https://nplus1.ru/news/2023/10/20/air-pollution-and-breast-cancer',
 'https://nplus1.ru/news/2023/10/20/voice-prediction-diabetes']

Первая ссылка ведет не совсем на новость, скорее, на объявление, поэтому давайте ее уберем:

In [8]:
news = news[1:] 

Теперь наша задача сводится к следующему: изучить одну страницу с новостью, научиться из нее вытаскивать текст и всю необходимую информацию, а потом применить весь набор действий к каждой ссылке из `news` в цикле. Посмотрим на новость с индексом 0, у вас может быть другая, новости обновляются.

In [9]:
link0 = news[0]
print(link0)

https://nplus1.ru/news/2023/10/18/beware-ppi


### Задача 1

Отправьте запрос к странице по ссылке `link0` с одной новостью, получите результат в виде объекта `BeautifulSoup` и сохраните его как `soup0`.

In [10]:
page0 = requests.get(link0)
soup0 = BeautifulSoup(page0.text)

### Задача 2

Найдите заголовок новости и сохраните его в переменную `title`.

In [11]:
title = soup0.find("title").text
print(title)

Связь противоязвенных препаратов с деменцией подтвердили в крупном исследовании. Оно охватило почти два миллиона человек за 19 лет


### Задача 3

Найдите имя автора новости и дату её публикации. Сохраните их в `author` и `date` соответственно.

In [12]:
date = soup0.find("meta", {"itemprop" : "datePublished"}).get("content") 
author = soup0.find("meta", {"name" : "author"}).get("content") 
print(date, author)

2023-10-18 Олег Лищук


### Задача 4

Найдите сложность новости и рубрики, к которым она относится. Сохраните сложность в переменную `diffc`. Рубрики сначала можно сохранить в список, а затем его элементы объединить в одну строку `rubs`.

**Подсказка:** чтобы упростить себе жизнь, найдите сначала раздел (`div`), в котором на странице хранится дата и время публикации, сложность и рубрики, а затем выполняйте поиск в рамках этого раздела.

In [13]:
div = soup0.find("div", 
                 {"class" : "flex flex-wrap lg:mb-10 gap-2 text-tags xl:pr-9"})
diffc = div.find_all("span")[3].text
print(diffc)

2.9


In [14]:
rubs_raw = div.find_all("span")[4:] 
rubs_L = [r.text for r in rubs_raw] 
rubs = ", ".join(rubs_L)
print(rubs)

Медицина


### Задача 5

Соберите из абзацев текст новости и сохраните его в переменную `text`. Избавьтесь от постронних символов (`\xa0`, `\n`) в тексте. 

In [15]:
# на занятии решили не избавляться от \n
# потом передумали – на этапе работы с датафреймом поправили

pars_raw = soup0.find_all("p", {"class" : "mb-6"}) 
pars = [p.text for p in pars_raw] 
text = " ".join(pars)
text = text.replace("\xa0", " ")

### Задача 6

Напишите функцию `get_news()`, которая принимает на вход ссылку на страницу с одной новостью, а возвращает список из следующих характеристик: имя автора, дата публикации, сложность новости, рубрики, текст новости.

In [16]:
# в условии пропущен заголовок, его тоже забираем

def get_news(link0):

    page0 = requests.get(link0)
    soup0 = BeautifulSoup(page0.text)
    title = soup0.find("title").text
    date = soup0.find("meta", {"itemprop" : "datePublished"}).get("content") 
    author = soup0.find("meta", {"name" : "author"}).get("content") 
    div = soup0.find("div", 
                 {"class" : "flex flex-wrap lg:mb-10 gap-2 text-tags xl:pr-9"})
    diffc = div.find_all("span")[3].text
    rubs_raw = div.find_all("span")[4:] 
    rubs_L = [r.text for r in rubs_raw] 
    rubs = ", ".join(rubs_L)
    pars_raw = soup0.find_all("p", {"class" : "mb-6"}) 
    pars = [p.text for p in pars_raw] 
    text = " ".join(pars)
    text = text.replace("\xa0", " ").replace("\n", " ")
    
    return [title, author, date, diffc, rubs, text]

Отлично! Теперь осталось применить функцию ко всем ссылкам в списке `news`. Чтобы сайт не понял, что мы его автоматически грабим, будем выгружать новости постепенно – с задержкой в 1.5 секунды. Импортируем для этого функцию `sleep` :

In [17]:
from time import sleep

Теперь будем применять функцию в цикле к каждой ссылке в `news`, только с одним дополнением – добавленной конструкцией `try-except`, которая позволит продолжать исполнение цикла, если при применении функции Python столкнулся с ошибкой любого вида:

In [18]:
info = []
for n in news:
    # пробуй исполнить следующий код
    try:
        res = get_news(n)
        info.append(res)
        print(n)
    # если он вызвал ошибку, печатай сообщение и иди дальше
    except:
        print("Something went wrong")
        print(n)
    sleep(1.2)

https://nplus1.ru/news/2023/10/18/beware-ppi
https://nplus1.ru/news/2023/10/21/frb-and-plasma
https://nplus1.ru/news/2023/10/21/cognitive-behavior-therapy
https://nplus1.ru/news/2023/10/21/pulsatile-hydrocortisone
https://nplus1.ru/news/2023/10/21/daily-steps-prior-to-surgery
https://nplus1.ru/news/2023/10/20/eye-ticking
https://nplus1.ru/news/2023/10/20/python-ate-python
https://nplus1.ru/news/2023/10/20/air-pollution-and-breast-cancer
https://nplus1.ru/news/2023/10/20/voice-prediction-diabetes
https://nplus1.ru/news/2023/10/20/janus-kinase-inhibitors-and-acne
https://nplus1.ru/news/2023/10/20/gorillas-inbreeding
https://nplus1.ru/news/2023/10/20/suzuki-moqba
https://nplus1.ru/news/2023/10/20/threshold-greenland-ice-shield
https://nplus1.ru/news/2023/10/20/neanderthal-ancestry
https://nplus1.ru/news/2023/10/20/snake-gun-from-orel
https://nplus1.ru/news/2023/10/20/come-on-digit-get-up
https://nplus1.ru/news/2023/10/19/mars-drone-again-wins
https://nplus1.ru/news/2023/10/19/io-juno
http

Посмотрим на несколько элементов `info`:

In [ ]:
info[10:12]

Финальный штрих – импортируем `pandas` и преобразуемый полученный список списков в датафрейм:

In [19]:
import pandas as pd

In [20]:
df = pd.DataFrame(info)
df.head()

,0,1,2,3,4,5
0,Связь противоязвенных препаратов с деменцией п...,Олег Лищук,2023-10-18,2.9,Медицина,Оно охватило почти два миллиона человек за 19 ...
1,Самый далекий быстрый радиовсплеск прошел скво...,Александр Войтюк,2023-10-21,4.9,Астрономия,Излучение шло до Земли около 8 миллиардов лет ...
2,Когнитивно-поведенческая терапия оказалась эфф...,Слава Гоменюк,2023-10-21,3.2,Медицина,Это показал метаанализ Систематический обзор и...
3,Подкожная помпа с кортизолом эффективно облегч...,Слава Гоменюк,2023-10-21,4.3,Медицина,В частности — эмоциональные нарушения Британск...
4,Семь с половиной тысяч шагов в день до операци...,Слава Гоменюк,2023-10-21,2.9,Медицина,Риск снизился на половину Согласно результатам...


Добавим содержательные названия столбцов и выгрузим датафрейм в файл:

In [21]:
df.columns = ["title", "author", "date", "diffc", "rubrics", "text"]
df.head()

,title,author,date,diffc,rubrics,text
0,Связь противоязвенных препаратов с деменцией п...,Олег Лищук,2023-10-18,2.9,Медицина,Оно охватило почти два миллиона человек за 19 ...
1,Самый далекий быстрый радиовсплеск прошел скво...,Александр Войтюк,2023-10-21,4.9,Астрономия,Излучение шло до Земли около 8 миллиардов лет ...
2,Когнитивно-поведенческая терапия оказалась эфф...,Слава Гоменюк,2023-10-21,3.2,Медицина,Это показал метаанализ Систематический обзор и...
3,Подкожная помпа с кортизолом эффективно облегч...,Слава Гоменюк,2023-10-21,4.3,Медицина,В частности — эмоциональные нарушения Британск...
4,Семь с половиной тысяч шагов в день до операци...,Слава Гоменюк,2023-10-21,2.9,Медицина,Риск снизился на половину Согласно результатам...


**Дополнение 1.**  В списке ссылок какие-то ссылки встретились более одного раза – избавимся от одинаковых строк в таблице:

In [22]:
# inplace = True – чтобы сохранить изменения в df
# более изящная альтернатива df = df.drop_duplicates()

df.drop_duplicates(inplace = True)

**Дополнение 2.** Избавимся от `/n`, которые все-таки закрались в тексты новостей. Напишем lambda-функцию, которая будет описывать действие для одной ячейки в столбце, и применим ее ко всему столбцу `text` с помощью метода `.apply()`: 

In [23]:
# x – наше условное название для содержимого одной ячейки
# функция с lambda без названия принимает на вход x (до :)
# и возвращает его же, но с исправленными символами (после :)
# метод .apply() «растягивает» функцию на все ячейки
# перезаписываем столбец text

df["text"] = df["text"].apply(lambda x: x.replace("\n", " "))

**Дополнение 3.** Исправим тип столбца со сложностью новости – сделаем его типом `float`, чтобы с элементами можно было работать как с числами.

In [24]:
df["diffc"] = df["diffc"].astype(float)

Раз сделали столбец числовым, значит, его можно использовать для сортировки! Отсортируем строки в датафрейме от самой сложной новости к самой простой:

In [26]:
df.sort_values("diffc", ascending = False)

Топ-3 самых сложных новостей:

In [27]:
df.sort_values("diffc", ascending = False).head(3)

,title,author,date,diffc,rubrics,text
24,Пространственная и временная координаты поменя...,Оксана Борзенкова,2023-10-18,8.5,,Этот подход может оказаться полезным не только...
45,Квантовые вычислители на нейтральных атомах за...,Оксана Борзенкова,2023-10-16,8.3,Физика,А еще они теперь могут исправлять ошибки во вр...
40,Антиматерия упала вниз. Гравитация Земли притя...,Дмитрий Рудик,2023-10-06,7.8,Физика,Гравитация Земли притягивает атомы антивеществ...


Топ-3 самых простых новостей:

In [28]:
df.sort_values("diffc").head(3)

,title,author,date,diffc,rubrics,text
32,Швейцарец обнаружил на морковном поле клад эпо...,Михаил Подрезов,2023-10-18,1.1,Археология,Возраст находок составляет около 3500 лет Швей...
6,Питон съел питона. Заживо,Катерина Петрова,2023-10-20,1.1,Зоология,"Заживо Ник Сток (Nick Stock), управляющий Авст..."
14,В Орле нашли пищаль XVII века. Она украшена де...,Михаил Подрезов,2023-10-20,1.2,Археология,Она украшена декором в виде змеиной головы Исс...


Наконец, выгружаем датафрейм в файл:

In [29]:
df.to_excel("nplus1.xlsx")